# 02 — GDELT News Signal (Monthly Volume + Tone)

**Goal:** Build two monthly news features from GDELT:
- `news_volume`: how many layoff-related news articles per month
- `news_tone`: average tone (sentiment) of those articles per month

**Output files (saved to repo):**
- `data/raw/gdelt/gdelt_news_volume_monthly.csv`
- `data/raw/gdelt/gdelt_news_tone_monthly.csv`
- `data/raw/gdelt/gdelt_news_monthly.csv` (merged)

**Time window:** July 2020 – June 2025 (aligned with our 5 fiscal years of CA WARN PDFs)

In [3]:
import os
import pandas as pd
import requests
from io import StringIO


In [4]:
# Where we save downloaded/processed GDELT outputs
OUT_DIR = "data/raw/gdelt"
os.makedirs(OUT_DIR, exist_ok=True)

print("Saving outputs to:", OUT_DIR)

Saving outputs to: data/raw/gdelt


## 1 - Define the news query

We use a **targeted layoff-related keyword query** to capture media coverage about workforce reductions.

We are **not filtering to California** because national/global layoff narratives can affect California layoffs.
(Our outcome is still CA WARN layoffs; this is a macro news signal.)

In [5]:
# Layoff-related keyword query
QUERY = '(layoffs OR "job cuts" OR "hiring freeze" OR downsizing OR "workforce reduction")'

# Aligned with WARN fiscal year PDFs: Jul 2020 -> Jun 2025
# YYYYMMDDHHMMSS format
START_DATETIME = "20200701000000"
END_DATETIME   = "20250630235959"

# GDELT endpoint (DOC 2.1 API)
BASE_URL = "https://api.gdeltproject.org/api/v2/doc/doc"

print("Query:", QUERY)
print("Date range:", START_DATETIME, "to", END_DATETIME)

Query: (layoffs OR "job cuts" OR "hiring freeze" OR downsizing OR "workforce reduction")
Date range: 20200701000000 to 20250630235959


## 2 - Pull two timeline series from GDELT

We make two API requests:

1) **TimelineVolRaw** → monthly article counts matching the query  
2) **TimelineTone** → monthly average tone for matching articles

We request `format=csv` so it is easy to load into pandas.

In [6]:
def fetch_gdelt_timeline_csv(mode: str) -> pd.DataFrame:
    """
    Fetch a GDELT timeline series as a pandas DataFrame.

    mode : str
        "timelinevolraw" for article counts, or "timelinetone" for average tone.

    Returns
   
    pd.DataFrame
        Raw GDELT CSV parsed into a DataFrame.
    """
    params = {
        "query": QUERY,
        "mode": mode,
        "format": "csv",
        "startdatetime": START_DATETIME,
        "enddatetime": END_DATETIME,
    }

    r = requests.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()  # will throw a helpful error if the request fails

    return pd.read_csv(StringIO(r.text))

In [7]:
vol_raw = fetch_gdelt_timeline_csv("timelinevolraw")
vol_raw.head()

,Date,Series,Value
0,2020-07-01,Article Count,3250
1,2020-07-01,Total Monitored Articles,483273
2,2020-07-02,Article Count,3018
3,2020-07-02,Total Monitored Articles,461931
4,2020-07-03,Article Count,2080


In [8]:
tone_raw = fetch_gdelt_timeline_csv("timelinetone")
tone_raw.head()

,Date,Series,Value
0,2020-07-01,Average Tone,-2.3281
1,2020-07-02,Average Tone,-2.1462
2,2020-07-03,Average Tone,-2.2157
3,2020-07-04,Average Tone,-2.8216
4,2020-07-05,Average Tone,-1.8916


In [9]:
print("Volume raw columns:", vol_raw.columns.tolist())
print("Tone raw columns:", tone_raw.columns.tolist())

Volume raw columns: ['Date', 'Series', 'Value']
Tone raw columns: ['Date', 'Series', 'Value']


## 3 - Standardize output

GDELT sometimes returns slightly different column names.
We standardize to:

- `date` (parsed as datetime)
- `month` (YYYY-MM-01)
- `news_volume` (sum per month)
- `news_tone` (mean per month)

In [12]:
def standardize_gdelt_timeline(df: pd.DataFrame, value_name: str, series_keep: str, agg: str) -> pd.DataFrame:
    """
    Standardize GDELT timeline output that has columns: Date, Series, Value.

    Parameters
    ----------
    df : pd.DataFrame
        Raw GDELT output.
    value_name : str
        Output value column name: 'news_volume' or 'news_tone'.
    series_keep : str
        Which Series row to keep, e.g. 'Article Count' or 'Average Tone'.
    agg : str
        'sum' for volume, 'mean' for tone.

    Returns
    -------
    pd.DataFrame with columns: month, {value_name}
    """
    df = df.copy()

    # 1) Keep only the series we want (prevents strings from entering numeric operations)
    if "Series" in df.columns:
        df = df[df["Series"] == series_keep].copy()

    # 2) Rename and convert types
    df = df.rename(columns={"Date": "date", "Value": value_name})
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Value must be numeric
    df[value_name] = pd.to_numeric(df[value_name], errors="coerce")

    df = df.dropna(subset=["date", value_name])

    # 3) Convert to month buckets
    df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

    # 4) Aggregate monthly
    if agg == "sum":
        out = df.groupby("month", as_index=False)[value_name].sum()
    elif agg == "mean":
        out = df.groupby("month", as_index=False)[value_name].mean()
    else:
        raise ValueError("agg must be 'sum' or 'mean'")

    return out

In [15]:
# Create monthly volume + tone

vol_m = standardize_gdelt_timeline(
    vol_raw, value_name="news_volume", series_keep="Article Count", agg="sum"
)


tone_m = standardize_gdelt_timeline(
    tone_raw, value_name="news_tone", series_keep="Average Tone", agg="mean"
)



print(vol_m.head())
print(tone_m.head())

       month  news_volume
0 2020-07-01        64201
1 2020-08-01        47418
2 2020-09-01        41715
3 2020-10-01        27663
4 2020-11-01        19858
       month  news_tone
0 2020-07-01  -2.080829
1 2020-08-01  -2.014797
2 2020-09-01  -1.978027
3 2020-10-01  -1.937167
4 2020-11-01  -1.985260


In [16]:
vol_raw["Series"].value_counts().head(10)

Series
Article Count               1808
Total Monitored Articles    1808
Name: count, dtype: int64

In [17]:
tone_raw["Series"].value_counts().head(10)

Series
Average Tone    1808
Name: count, dtype: int64

In [18]:
# Merge into one table + sanity check

gdelt_monthly = pd.merge(vol_m, tone_m, on="month", how="outer").sort_values("month")
gdelt_monthly.head()

,month,news_volume,news_tone
0,2020-07-01,64201,-2.080829
1,2020-08-01,47418,-2.014797
2,2020-09-01,41715,-1.978027
3,2020-10-01,27663,-1.937167
4,2020-11-01,19858,-1.985260


## 4 - Save outputs

We save:
- volume monthly CSV
- tone monthly CSV
- merged CSV (used later when building master table)

In [19]:
vol_path = os.path.join(OUT_DIR, "gdelt_news_volume_monthly.csv")
tone_path = os.path.join(OUT_DIR, "gdelt_news_tone_monthly.csv")
merged_path = os.path.join(OUT_DIR, "gdelt_news_monthly.csv")

vol_m.to_csv(vol_path, index=False)
tone_m.to_csv(tone_path, index=False)
gdelt_monthly.to_csv(merged_path, index=False)

print("Saved:", vol_path)
print("Saved:", tone_path)
print("Saved:", merged_path)

Saved: data/raw/gdelt/gdelt_news_volume_monthly.csv
Saved: data/raw/gdelt/gdelt_news_tone_monthly.csv
Saved: data/raw/gdelt/gdelt_news_monthly.csv


In [20]:
gdelt_monthly.head(10)

,month,news_volume,news_tone
0,2020-07-01,64201,-2.080829
1,2020-08-01,47418,-2.014797
2,2020-09-01,41715,-1.978027
3,2020-10-01,27663,-1.937167
4,2020-11-01,19858,-1.985260
5,2020-12-01,28134,-1.608994
6,2021-01-01,25409,-1.594477
7,2021-02-01,24282,-1.546736
8,2021-03-01,22878,-1.490381
9,2021-04-01,18249,-1.646990


In [21]:
# news = pd.read_csv("data/raw/gdelt/gdelt_news_monthly.csv", parse_dates=["month"])
# master = master.merge(news, on="month", how="left")